<a href="https://colab.research.google.com/github/Droganderon/Colab/blob/main/Lab%2001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Лабораторная работа №1. "Разработка классификатора методом k ближайших соседей"

In [27]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


1. Загрузить данные из файла heart.csv, используя функцию read_csv() библиотеки pandas

In [28]:
df = pd.read_csv('/content/drive/MyDrive/diabetes.csv')

2. Вывести несколько строк датафрейма. Более подробная информация о датасете https://www.kaggle.com/mathchi/diabetes-data-set

In [38]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


3. Проверить сбалансированность классов. Можно воспользоваться методом value_counts() датафрейма. Для каждого класса нужно вывести долю примеров от общего числа

In [40]:
df['Outcome'].value_counts()/df.shape[0]

0    0.651042
1    0.348958
Name: Outcome, dtype: float64

Class 0: 0.651
Class 1: 0.349


4. Провести процедуру нормирования признаков. Вариант 1: так, чтобы все признаки принимали значения в диапазоне $[0;1]$, используя преобразование $X^{*}=(X-X_{min})/{\Delta{X}}$ для каждого признака. Вариант 2: так, чтобы все признаки принимали значения в диапазоне $[-3\sigma;+3\sigma]$, используя преобразование $X^{*}=(X-\mu_{X})/{\sigma_{X}}$ для каждого признака.
Процедуру обезразмеривания оформить в виде функции

In [32]:
def normalization(X):
  return (X)/(X.abs().max())
df_norm = df.apply(normalization)
df_norm

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,0.352941,0.743719,0.590164,0.353535,0.000000,0.500745,0.259091,0.617284,1.0
1,0.058824,0.427136,0.540984,0.292929,0.000000,0.396423,0.145041,0.382716,0.0
2,0.470588,0.919598,0.524590,0.000000,0.000000,0.347243,0.277686,0.395062,1.0
3,0.058824,0.447236,0.540984,0.232323,0.111111,0.418778,0.069008,0.259259,0.0
4,0.000000,0.688442,0.327869,0.353535,0.198582,0.642325,0.945455,0.407407,1.0
...,...,...,...,...,...,...,...,...,...
763,0.588235,0.507538,0.622951,0.484848,0.212766,0.490313,0.070661,0.777778,0.0
764,0.117647,0.613065,0.573770,0.272727,0.000000,0.548435,0.140496,0.333333,0.0
765,0.294118,0.608040,0.590164,0.232323,0.132388,0.390462,0.101240,0.370370,0.0
766,0.058824,0.633166,0.491803,0.000000,0.000000,0.448584,0.144215,0.580247,1.0


In [ ]:
def normalization(X):


In [ ]:
X_norm = normalization(data.drop('Outcome', axis = 1))
Y = data.Outcome

In [ ]:
X_norm

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.639530,0.847771,0.149543,0.906679,-0.692439,0.203880,0.468187,1.425067
1,-0.844335,-1.122665,-0.160441,0.530556,-0.692439,-0.683976,-0.364823,-0.190548
2,1.233077,1.942458,-0.263769,-1.287373,-0.692439,-1.102537,0.604004,-0.105515
3,-0.844335,-0.997558,-0.160441,0.154433,0.123221,-0.493721,-0.920163,-1.040871
4,-1.141108,0.503727,-1.503707,0.906679,0.765337,1.408828,5.481337,-0.020483
...,...,...,...,...,...,...,...,...
763,1.826623,-0.622237,0.356200,1.721613,0.869464,0.115094,-0.908090,2.530487
764,-0.547562,0.034575,0.046215,0.405181,-0.692439,0.609757,-0.398023,-0.530677
765,0.342757,0.003299,0.149543,0.154433,0.279412,-0.734711,-0.684747,-0.275580
766,-0.844335,0.159683,-0.470426,-1.287373,-0.692439,-0.240048,-0.370859,1.169970


5. Разделить выборку на три части: обучающую, валидационную и тестовую в соотношении 0.6:0.2:0.2. Выбор из исходной выборки должен осуществляться случайным образом. Можно воспользоваться функцией choice() библиотеки numpy.random. Можно также воспользоваться специальной функцией train_test_split() библиотеки sklearn.model_selection. Проверить сбалансированность классов в полученных выборках.

In [36]:
from sklearn.model_selection import train_test_split
Y = df.Outcome
X_train, X_test, Y_train, Y_test = train_test_split(df_norm, Y, test_size=0.4, random_state=42)
X_test, X_val, Y_test, Y_val = train_test_split(X_test, Y_test, test_size=0.5, random_state=42)
Y_train.value_counts()/Y_train.shape[0], Y_test.value_counts()/Y_test.shape[0], Y_val.value_counts()/Y_val.shape[0]

(0    0.63913
 1    0.36087
 Name: Outcome, dtype: float64,
 0    0.675325
 1    0.324675
 Name: Outcome, dtype: float64,
 0    0.662338
 1    0.337662
 Name: Outcome, dtype: float64)

In [ ]:
from sklearn.model_selection import train_test_split



Class 0: 0.643
Class 1: 0.357

Class 0: 0.675
Class 1: 0.325

Class 0: 0.649
Class 1: 0.351



6. Написать функцию knn_classification(X,y,X_new,k), реализующую метод k-ближайших соседей для задачи классификации. Параметрами функции являются: X,y - обучающая выборка, X_new - массив (или датафрейм) признаков для новых объектов, k - гиперпараметр метода (кол-во ближайших соседей, в функции предусмотреть возможность задания k по умолчанию). Функция должна возвращать прогноз для новых объектов в виде массива классов (y_pred).

In [ ]:
def knn(X, Y, X_new, k):

    return Y_pred


7. Написать функцию для расчета метрик качества модели классификации. Функция должна возвращать значения оценок Precision, Recall, F1-score.

In [ ]:
def fscore(y, y_pred):

    return prec, rec, f1_score, accuracy

8. Выполнить предсказания классов для валидационной выборки для произвольного гиперпараметра k, рассчитать и вывести метрики Precision, Recall, F1-score и Accuracy.

In [ ]:
Y_val_pred = knn(X_train, Y_train, X_val, k=2)
prec, recall, f1_score, accuracy =


prec: 0.792, recall: 0.380, f1_score: 0.514, accuracy: 0.766


9. Задать массив или список значений гиперпараметра k (например, от 2 до 10 с шагом 2). Провести процедуру валидации, т.е. подбора оптимального значения k по критерию максимума точности. Для оценки точности использовать F1-score метрику. Выводить для каждого значения k полученное значение f1 score. В конце вывести лучший результат.

In [ ]:
K = [i for i in range(2,10,2)]

for k in K:


k: 2, f1 score: 0.5135135135135135
k: 4, f1 score: 0.5066666666666666
k: 6, f1 score: 0.5609756097560976
k: 8, f1 score: 0.5121951219512195

Best k: 6, best val f1 score: 0.5609756097560976


10. Рассчитать оценки точности (accuracy, precision, recal, F1-score) классификатора для тестовой выборки с оптимальным значением гиперпараметра k. Сделать вывод о качестве классификатора.

prec: 0.683, recall: 0.519, f1_score: 0.589, accuracy: 0.747


11. Сравнить результаты с функцией KNeighborsClassifier из библиотеки sklearn. Если получился другой результат подумать в чем дело.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=best_k, metric='euclidean')
clf.fit(X_train, Y_train)
Y_test_pred = clf.predict(X_test)


prec: 0.683, recall: 0.519, f1_score: 0.589, accuracy: 0.747
